<a href="https://colab.research.google.com/github/MSISCapstone/CSPatchTST/blob/mouryavs/WPatchTST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone -b mouryavs https://github.com/MSISCapstone/CSPatchTST.git

Cloning into 'CSPatchTST'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 667 (delta 58), reused 52 (delta 38), pack-reused 578 (from 1)
Receiving objects: 100% (667/667), 42.11 MiB | 26.92 MiB/s, done.
Resolving deltas: 100% (336/336), done.
Filtering content: 100% (2/2), 221.27 MiB | 45.14 MiB/s, done.


In [ ]:
cd /content/CSPatchTST/WPatchTST


/content/CSPatchTST/WPatchTST


In [ ]:
ls

environment.yml  requirements.txt  WPatchTST_Environment.ipynb


In [ ]:
%pip install -r requirements.txt



In [ ]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [ ]:
%cd /content/CSPatchTST
!ls


/content/CSPatchTST
datasets  LICENSE    PatchTST_self_supervised  pic	  WPatchTST
docs	  notebooks  PatchTST_supervised       README.md


In [ ]:
!grep -R "patch_len" -n . | head -n 25


./PatchTST_supervised/scripts/PatchTST/etth2.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/electricity.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/ettm1.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:23:patch_lengths="6,12,24"      # Three temporal scales
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:43:      --patch_lengths $patch_lengths \
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:53:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/etth1.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/ettm2.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/univariate/etth2.sh:43:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/univariate/ettm1.sh:43:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/univariate/etth1.sh:43:      --patch_len 16\
./PatchTST_supervised/scripts/Pa

In [ ]:
!sed -n '1,120p' PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


#!/bin/bash

# Multi-Scale + Cross-Channel PatchTST for Weather Forecasting
# Combines both enhancements for maximum performance:
#   - Multi-scale: Captures temporal patterns at 6hr, 12hr, 24hr scales
#   - Cross-channel: Enables interaction between weather variables

model_name=PatchTST
random_seed=2021

root_path_name=./dataset/
data_path_name=weather.csv
model_id_name=weather
data_name=custom

seq_len=336

# Enable BOTH enhancements
multi_scale=1
channel_independent=0        # 0 = cross-channel (variables interact)

# Multi-scale configuration
patch_lengths="6,12,24"      # Three temporal scales
patch_strides="3,6,12"       # 50% overlap for each
patch_weights="0.2,0.5,0.3"  # Balanced fusion

for pred_len in 96 192 336 720
do
    python -u run_longExp.py \
      --random_seed $random_seed \
      --is_training 1 \
      --root_path $root_path_name \
      --data_path $data_path_name \
      --model_id $model_id_name'_multiCross_'$seq_len'_'$pred_len \
      --model $model_name \
 

In [ ]:
!cp PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh \
    PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh.bak


In [ ]:
!grep -R "python -u" -n PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


29:    python -u run_longExp.py \


In [ ]:
!grep -R "argparse" -n PatchTST_supervised | head -n 20


PatchTST_supervised/Formers/Pyraformer/single_step_main.py:1:import argparse
PatchTST_supervised/Formers/Pyraformer/single_step_main.py:241:    parser = argparse.ArgumentParser()
PatchTST_supervised/Formers/Pyraformer/long_range_main.py:1:import argparse
PatchTST_supervised/Formers/Pyraformer/long_range_main.py:311:    parser = argparse.ArgumentParser()
PatchTST_supervised/Formers/Pyraformer/pyraformer/graph_attention.py:12:import argparse
PatchTST_supervised/Formers/Pyraformer/pyraformer/graph_attention.py:428:    parser = argparse.ArgumentParser(description='Needed for graph self attention.')
PatchTST_supervised/Formers/FEDformer/run.py:1:import argparse
PatchTST_supervised/Formers/FEDformer/run.py:13:parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')
PatchTST_supervised/run_longExp.py:1:import argparse
PatchTST_supervised/run_longExp.py:9:    parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Tim

In [ ]:
!sed -n '1,200p' PatchTST_supervised/run_longExp.py


import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

    # random seed
    parser.add_argument('--random_seed', type=int, default=2021, help='random seed')

    # basic config
    parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
    parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
    parser.add_argument('--model', type=str, required=True, default='Autoformer',
                        help='model name, options: [Autoformer, Informer, Transformer]')

    # data loader
    parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
    parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
    parser.add_argument('--data_path', type=str,

In [ ]:
!grep -R "unfold(" -n PatchTST_supervised | head -n 20


PatchTST_supervised/layers/PatchTST_backbone.py:78:        z = z.unfold(dimension=-1, size=self.patch_len, step=self.stride)                   # z: [bs x nvars x patch_num x patch_len]


In [ ]:
!grep -R "PatchTST_backbone" -n PatchTST_supervised | head -n 20


PatchTST_supervised/models/PatchTST.py:11:from layers.PatchTST_backbone import PatchTST_backbone
PatchTST_supervised/models/PatchTST.py:40:            encoder = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:194:                self.model_trend = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
PatchTST_supervised/models/PatchTST.py:202:                self.model_res = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
PatchTST_supervised/models/PatchTST.py:211:                self.model = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
PatchTST_supervised/layers/PatchTST_backbone.py:1:__all__ = ['PatchTST_backbone']
PatchTST_supervised/layers/PatchTST_backbone.py:16:class PatchTST_backbone(nn.Module):


In [ ]:
!sed -n '1,120p' PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


#!/bin/bash

# Multi-Scale + Cross-Channel PatchTST for Weather Forecasting
# Combines both enhancements for maximum performance:
#   - Multi-scale: Captures temporal patterns at 6hr, 12hr, 24hr scales
#   - Cross-channel: Enables interaction between weather variables

model_name=PatchTST
random_seed=2021

root_path_name=./dataset/
data_path_name=weather.csv
model_id_name=weather
data_name=custom

seq_len=336

# Enable BOTH enhancements
multi_scale=1
channel_independent=0        # 0 = cross-channel (variables interact)

# Multi-scale configuration
patch_lengths="6,12,24"      # Three temporal scales
patch_strides="3,6,12"       # 50% overlap for each
patch_weights="0.2,0.5,0.3"  # Balanced fusion

for pred_len in 96 192 336 720
do
    python -u run_longExp.py \
      --random_seed $random_seed \
      --is_training 1 \
      --root_path $root_path_name \
      --data_path $data_path_name \
      --model_id $model_id_name'_multiCross_'$seq_len'_'$pred_len \
      --model $model_name \
 

In [ ]:
!grep -R "PatchTST_backbone" -n PatchTST_supervised | head -n 30


PatchTST_supervised/models/PatchTST.py:11:from layers.PatchTST_backbone import PatchTST_backbone
PatchTST_supervised/models/PatchTST.py:40:            encoder = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:194:                self.model_trend = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
PatchTST_supervised/models/PatchTST.py:202:                self.model_res = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
PatchTST_supervised/models/PatchTST.py:211:                self.model = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
PatchTST_supervised/layers/PatchTST_backbone.py:1:__all__ = ['PatchTST_backbone']
PatchTST_supervised/layers/PatchTST_backbone.py:16:class PatchTST_backbone(nn.Module):
